<a href="https://colab.research.google.com/github/lalonardoni/treinamento-ia-gen/blob/main/Parte3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Dependências*

Fontes:
https://python.langchain.com/v0.1/docs/use_cases/chatbots/quickstart/


In [1]:
import getpass
import os

In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


Create LLM

In [3]:
%pip install --upgrade --quiet langchain-google-genai langchain langchain-community langchain-chroma beautifulsoup4

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
chat = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
loader = WebBaseLoader("https://www.cptec.inpe.br/previsao-tempo/pr/maringa")

data = loader.load()

In [7]:
# # Iterando sobre os documentos carregados para ver o conteúdo
# for doc in data:
#     print(doc.page_content)

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)

In [9]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)

In [10]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("o que é o evento da uem secomp?")

docs

[Document(metadata={'description': 'Centro de Previsão de Tempo e Estudos Climáticos - CPTEC/INPE', 'language': 'pt_BR', 'source': 'https://www.cptec.inpe.br/previsao-tempo/pr/maringa', 'title': 'Centro de Previsão de Tempo e Estudos Climáticos - INPE\n                 - Maringá / PR\n        '}, page_content='Sobre\n\n Sobre o CPTEC \nQuem é quem\n Pesq. e Desenvolvimento \n Biblioteca \nPós - Graduação\nSupercomputação\n Perg. Frequentes\n Ouvidoria\n Links Indicados\n Serviços ao Usuário (pdf)\n\n\n\nProdutos\n\n Previsão de Tempo \n Imagens de Satélites \n Aeroportos \nPrecipitação por Radar \n Queimadas \n Radiação UV\n infoClima\n El Niño e La Niña \n Precipitação por Satélite \n Plat. Coleta de Dados\n Fenômenos Naturais \n\n\n\nPesquisa e Desenvolvimento\n\n Cenapad \n ProVeg - FAPESP - (Parceria CPTEC/OBT) \n Projeta'),
 Document(metadata={'description': 'Centro de Previsão de Tempo e Estudos Climáticos - CPTEC/INPE', 'language': 'pt_BR', 'source': 'https://www.cptec.inpe.br/p

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [14]:
from langchain.memory import ChatMessageHistory

demo_ephemeral_chat_history = ChatMessageHistory()

demo_ephemeral_chat_history.add_user_message("qual a previsão do tempo para 27/10?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

'O texto fornecido não contém informações sobre a previsão do tempo para 27/10. Ele descreve a estrutura do site do CPTEC/INPE, mas não fornece previsões meteorológicas específicas. \n\nPara obter a previsão do tempo para 27/10, recomendo que você acesse o site do CPTEC/INPE e procure a seção de "Previsão de Tempo". \n'

In [42]:
demo_ephemeral_chat_history.add_user_message("de que ano?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

'O texto fala sobre a SECOMP 2023 e convida para a SECOMP 2024.  Para saber os dias específicos da SECOMP, você precisa especificar se quer saber os dias da SECOMP 2023 ou da SECOMP 2024. \n'

In [45]:
demo_ephemeral_chat_history.add_user_message("qual é o cronograma do evento?")

document_chain.invoke(
    {
        "messages": demo_ephemeral_chat_history.messages,
        "context": docs,
    }
)

'O texto fornecido não contém informações sobre o cronograma do evento SECOMP. \n'